# Wstęp do Uczenia Maszynowego - Lab 5

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.random.seed = 42

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

In [ ]:
y = np.array(data['chd'])
X = data.drop(['chd'],axis=1)

In [ ]:
map_dict = {'Present': 1, 'Absent':0}
X['famhist'] = X['famhist'].map(map_dict)
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

## Ensemble Methods
Na potrzeby stosowania różnych metod Ensemble Learningu załadujemy sobie już 3 modele z których będziemy potem korzystać.

In [ ]:
model1 = DecisionTreeClassifier(random_state=1)
model2 = KNeighborsClassifier()
model3 = LogisticRegression(random_state=1, max_iter=1000)
estimators=[('DecisionTree', model1), ('KNN', model2), ('LR', model2)]

### Max Voting
lub Hard Voting - głosowanie większościowe



In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
model = VotingClassifier(estimators=estimators, voting='hard')
model.fit(X_train,y_train)

y_hat = model.predict(X_test)
accuracy_score(y_test, y_hat), model.score(X_test,y_test)

### Averaging
Soft Voting. Nie patrzymy na liczbę głosów, ale na "pewność". Patrzymy na ile pewny jest klasyfikator, że rekord należy do klasy 1. c1 - 90%, c2 - 49%, c3 - 49%

* hard voting: klasa 0 (1 głos na tak, 2 głosy na nie)
* soft voting: klasa 1 ( (90 + 49 + 49)/3 $\approx$ 63% > 50%)


In [ ]:
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

pred1=model1.predict_proba(X_test)
pred2=model2.predict_proba(X_test)
pred3=model3.predict_proba(X_test)

# Average
pred_average=(pred1+pred2+pred3)/3
y_hat = np.argmax(pred_average, axis=1)
print(accuracy_score(y_test, y_hat))

# Weighted Average
pred_weighted_average=(pred1*0.05+pred2*0.05+pred3*0.9)
y_hat = np.argmax(pred_weighted_average, axis=1)
print(accuracy_score(y_test, y_hat))

### Stacking
![Stacking](https://miro.medium.com/max/700/1*RP0pkQEOSrw9_EjFu4w3gg.png)

Bierzemy kilka różnych modeli (base) i jeden meta-model. Meta-model uczy się przewidzieć wynik na podstawie wyników z base.

Zasada kciuka: ostatni model jest raczej prosty (regresja liniowa/logistyczna).
Więcej (np. o trenowaniu) tutaj: https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/



In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
from sklearn.model_selection import train_test_split
clf.fit(X_train, y_train).score(X_test, y_test)

#### Czy zestackowanie kilku takich samych modeli zwiększy ich dokładność?
- Jeżeli tak to podaj przykład?
- Jeżeli nie to czy masz jakiś pomysł żeby ulepszyć tą metodę?

### Bagging
Bootstraping - to technika próbkowania, w której tworzymy podzbiory (próby) obserwacji z oryginalnego datasetu, **ze zwracaniem**. Rozmiar podzbiorów jest taki sam jak rozmiar oryginalnego datasetu.

1. Losujemy N **bootstrapowych** prób ze zbioru treningowego
2. Trenujemy niezależnie N "słabych" klasyfikatorów
3. Składamy wyniki "słabych" modeli 
    - **Klasyfikacja:** reguła większościowa / uśrednione prawdopodobieństwo
    - **Regresja:** Uśrednione wartości

In [ ]:
clf = BaggingClassifier(base_estimator=model1,
                        n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

#### Jakie widzicie wady i zalety takiej metody?

#### Random Forest
Najbardziej popularny algorytm Baggingowy.

Cechy:
- podstawowym algorytmem jest Drzewo Decyzyjne (wszystkie zalety drzew: obsługa NA)
- Do podziału każdego węzła wykorzystujemy losowe zmienne (ilość można wybrać jako hiperparametr)
- wbudowana metoda istotności zmiennych

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_rf = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                  max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                  min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                  max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                  random_state=0,
                                  n_jobs = -1)
model_rf.fit(X_train, y_train)
model_rf.score(X_test,y_test)

![Bagging vs Boosting](https://quantdare.com/wp-content/uploads/2016/04/bb3.png)

### Boosting
Boosting działa podobnie jak Bagging z jedną różnicą. Każda kolejna próba bootstrap jest tworzona w taki sposób, że losuje z większym prawdopodobieństwiem obserwacje **źle sklasyfikowane**. W skrócie: Boosting uczy się na błędach, które popełnił w poprzednich iteracjach.

#### AdaBoost
Najprostsza metoda boostingowa

W każdej iteracji dla nowego klasyfikatora $h$ dobiera $\alpha_t$ minimalizujące
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/3ad2646753bae9d7170c88485eec7db956feecc1)
Gdzie $F_{t-1}$ - boosted classifier z poprzedniej iteracji

W każdej iteracji wagi samplowania dla każdego punktu $x_i$:
$$E(F_{t-1}(x_i))$$

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
model = AdaBoostClassifier(random_state=1)
model.fit(X_train, y_train)
model.score(X_test,y_test)

#### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model= GradientBoostingClassifier(random_state=1,
                                  learning_rate=0.01)
model.fit(X_train, y_train)
model.score(X_test,y_test)

#### XGBoost
Zaawansowana implementacja Gradient Boostingu

In [ ]:
from xgboost import XGBClassifier # Inna paczka niż sklearn!

In [ ]:
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    nround = 100, # Ilość itereacji boosingowych
                    max_depth=4 # Maksymalna głębokość drzewa 
                    )
model.fit(X_train, y_train)
model.score(X_test,y_test)